<a href="https://colab.research.google.com/github/rvaldivia95/AI-Udacity-Nanodegree/blob/master/SudokuMyVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
rows = 'ABCDEFGHI'
cols = '123456789'
 

def cross(a, b):
    return [s+t for s in a for t in b]

boxes = cross(rows, cols)

row_units = [cross(r, cols) for r in rows]
column_units = [cross(rows, c) for c in cols]
square_units = [cross(rs, cs) for rs in ('ABC','DEF','GHI') for cs in ('123','456','789')]
unitlist = row_units + column_units + square_units
units = dict((s, [u for u in unitlist if s in u]) for s in boxes)
peers = dict((s, set(sum(units[s],[]))-set([s])) for s in boxes)

def display(values):
    """
    Display the values as a 2-D grid.
    Input: The sudoku in dictionary form
    Output: None
    """
    width = 1+max(len(values[s]) for s in boxes)
    line = '+'.join(['-'*(width*3)]*3)
    for r in rows:
        print(''.join(values[r+c].center(width)+('|' if c in '36' else '')
                      for c in cols))
        if r in 'CF': print(line)
    return

def grid_values(grid):
    """Convert grid string into {<box>: <value>} dict with '.' value for empties.

    Args:
        grid: Sudoku grid in string form, 81 characters long
    Returns:
        Sudoku grid in dictionary form:
        - keys: Box labels, e.g. 'A1'
        - values: Value in corresponding box, e.g. '8', or '.' if it is empty.
    """
    sudoku = {}
    
    for n in range(0,len(grid)):
      if grid[n] == '.':

        sudoku[boxes[n]] = '123456789'

      else:

        sudoku[boxes[n]] = grid[n]

    return sudoku

def eliminate(values):

  for ru in row_units:

      for n in range(0,len(ru)):
        
        if len(values[ru[n]]) == 1:

          for i in range(0,len(ru)):

            if len(values[ru[i]]) != 1:
          
              values[ru[i]]= values[ru[i]].replace(values[ru[n]],'')

            else:

              pass

  for su in square_units:

      for n in range(0,len(su)):
        
        if len(values[su[n]]) == 1:

          for i in range(0,len(su)):

            if len(values[su[i]]) != 1:
          
              values[su[i]]= values[su[i]].replace(values[su[n]],'')

            else:

              pass

  for cu in column_units:

      for n in range(0,len(cu)):
        
        if len(values[cu[n]]) == 1:

          for i in range(0,len(cu)):

            if len(values[cu[i]]) != 1:
          
              values[cu[i]]= values[cu[i]].replace(values[cu[n]],'')

            else:

              pass

  return values

 #   Udacity solution 

 #   solved_values = [box for box in values.keys() if len(values[box]) == 1]
 #   for box in solved_values:
 #       digit = values[box]
 #       for peer in peers[box]:
 #           values[peer] = values[peer].replace(digit,'')
 #   return values



def only_choice(values):

    for unit in unitlist:
        for digit in '123456789':
            dplaces = [box for box in unit if digit in values[box]]
            if len(dplaces) == 1:
                values[dplaces[0]] = digit
                
    return values

def reduce_puzzle(values):
    stalled = False
    while not stalled:

        # Check how many boxes have a determined value
        solved_values_before = len([box for box in values.keys() if len(values[box]) == 1])

  
        values = eliminate(values)
        values = only_choice(values)

        # Check how many boxes have a determined value, to compare
        solved_values_after = len([box for box in values.keys() if len(values[box]) == 1])
        # If no new values were added, stop the loop.
        stalled = solved_values_before == solved_values_after
        # Sanity check, return False if there is a box with zero available values:
        if len([box for box in values.keys() if len(values[box]) == 0]):
            return False

    return values

def search(values):
    "Using depth-first search and propagation, create a search tree and solve the sudoku."
    # First, reduce the puzzle using the previous function
    values = reduce_puzzle(values)
    
    if values is False:
      return False

    if all(len(values[s]) == 1 for s in boxes):

      return values

    n,s = min((len(values[s]), s) for s in boxes if len(values[s])>1)

    for value in values[s]:
      new_values = values.copy()
      new_values[s]=value
      attempt = search(new_values)
      
      if attempt:

        return attempt

      
grid = '4.....8.5.3..........7......2.....6.....8.4......1.......6.3.7.5..2.....1.4......'
values = grid_values(grid)
values = search(values)
display(values)






4 1 7 |3 6 9 |8 2 5 
6 3 2 |1 5 8 |7 9 4 
9 5 8 |7 2 4 |6 1 3 
------+------+------
8 2 5 |4 3 7 |1 6 9 
3 9 1 |5 8 6 |4 5 2 
7 4 6 |9 1 2 |3 8 7 
------+------+------
2 8 9 |6 4 3 |5 7 1 
5 6 3 |2 7 1 |9 4 8 
1 7 4 |8 9 5 |2 3 6 
